## Telco Churn Project Notebook

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
import sklearn.impute
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import env
import acquire

In [2]:
telco = acquire.get_telco_data()

In [ ]:
telco.head()

In [ ]:
telco.info()

In [ ]:
telco.describe()

In [3]:
# total_charges should be in the above .describe(), but it is the wrong dtype
telco.replace(to_replace = " ", value = np.nan, inplace = True)

In [4]:
telco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

In [5]:
# after attempts to change total_charges into a float, we discovered there is some empty space, so once we 
# replace those, we can now see a few missing values there, and we can change the dtype

# we will drop the 0 values from the data set, because we have plenty of data points for our analysis, and
# these customers haven't even had a chance to churn yet.

telco.dropna(axis = 0, inplace = True)

# now we change the dtype

telco.total_charges = telco.total_charges.astype('float')

In [ ]:
# we'll rerun the .describe()

telco.describe()

In [ ]:
telco.head()

In [6]:
# We want to keep the customer_id, but we'd like to 'take it out' of the data for future scaling,
# exploration, etc. So, we will set the index with the customer_id

# set index to customer_id:

telco.set_index('customer_id', inplace=True)

In [ ]:
telco.head()

In [7]:
telco['tenure_years'] = telco.tenure/12
telco.head()

,payment_type_id,internet_service_type_id,contract_type_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,...,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,tenure_years
customer_id,,,,,,,,,,,,,,,,,,,,,
0003-MKNFE,2,1,1,Male,0,No,No,9,Yes,Yes,...,No,Yes,No,59.9,542.40,No,Month-to-month,DSL,Mailed check,0.750000
0013-MHZWF,4,1,1,Female,0,No,Yes,9,Yes,No,...,Yes,Yes,Yes,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic),0.750000
0015-UOCOJ,1,1,1,Female,1,No,No,7,Yes,No,...,No,No,Yes,48.2,340.35,No,Month-to-month,DSL,Electronic check,0.583333
0023-HGHWL,1,1,1,Male,1,No,No,1,No,No phone service,...,No,No,Yes,25.1,25.10,Yes,Month-to-month,DSL,Electronic check,0.083333
0032-PGELS,3,1,1,Female,0,Yes,Yes,1,No,No phone service,...,No,No,No,30.5,30.50,Yes,Month-to-month,DSL,Bank transfer (automatic),0.083333


In [ ]:
corr = telco.corr()

plt.figure(figsize=(16,9))

ax = sns.heatmap(
    corr,
    annot = True,
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [8]:
# we have a ton of variables, so we're looking to combine a few columns into single 'encoded' variables
# here we're looking at whether a customer has phone service at all, and if they do do they have 1 or more lines
# so we'll put a 0 for none, 1 for yes, but 1 line, 2 for yes and more than 1 line

phone_service = [0 if i == 'No' else 1 for i in telco.phone_service]
multiple_lines = [1 if i == 'Yes' else 0 for i in telco.multiple_lines]

telco['phone_and_multi_line'] = [phone_service[i] + multiple_lines[i] for i in range(len(phone_service))]

In [9]:
# here we're addressing a similar issue to the phone and multi-line columns. We will look at the dependent and
# partner columns. With these we'll address them similar to before, 0 for 'no and no', 1 for 'yes and no', 2 for
# 'no and yes', and 3 for 'yes and yes'.

# we add all of that to and empty list, then turn that into a new column

partner_and_dependents = []

for i in range(len(telco.partner)):
    if telco.partner[i] == 'No' and telco.dependents[i] == 'No':
        partner_and_dependents.append(0)
    elif telco.partner[i] == 'Yes' and telco.dependents[i] == 'No':
        partner_and_dependents.append(1)
    elif telco.partner[i] == "No" and telco.dependents[i] == 'Yes':
        partner_and_dependents.append(2)
    elif telco.partner[i] == 'Yes' and telco.dependents[i] == 'Yes':
        partner_and_dependents.append(3)

telco['partner_and_dependents'] = partner_and_dependents

In [ ]:
telco.head(10)

In [ ]:
corr = telco.corr()

plt.figure(figsize=(16,9))

ax = sns.heatmap(
    corr,
    annot = True,
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [10]:
# we feel that dropping the type_id fields for payment, internet_service, and contract becuase these are
# artifacts of the joining process during SQL pull

telco = telco.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id'])

In [ ]:
telco.head()

In [11]:
# we're drop the columns partner, dependents, phone_service, tenure, and multiple_lines, because we've added 'encoded'
# versions on the dataframe

telco = telco.drop(columns=['partner', 'dependents', 'phone_service', 'tenure', 'multiple_lines'])

In [ ]:
telco.sample(5)

In [12]:
# now we need to encode our data for numerous columns: payment, internet_service, and contract_type
# we will use the one hot encoder because we're not 'ranking' the different options.
# they are categorical in nature

one_hot = OneHotEncoder(categories = 'auto', sparse = False)
payment_encoded = one_hot.fit_transform(telco[['payment_type']])

labels = list(np.array(telco.payment_type.value_counts().index))

payment_encoded_df = pd.DataFrame(payment_encoded, columns = labels, index = telco.index)

In [ ]:
payment_encoded_df.head()

In [13]:
# next we'll one hot encode the internet service

one_hot = OneHotEncoder(categories = 'auto', sparse = False)
internet_encoded = one_hot.fit_transform(telco[['internet_service_type']])

labels = list(telco.internet_service_type.value_counts().sort_index().index)

internet_encoded_df = pd.DataFrame(internet_encoded, columns = labels, index = telco.index)

In [ ]:
internet_encoded_df.head()

In [14]:
# last we'll encode the contract type

one_hot = OneHotEncoder(categories = 'auto', sparse = False)
contract_encoded = one_hot.fit_transform(telco[['contract_type']])

labels = list(telco.contract_type.value_counts().sort_index().index)

contract_encoded_df = pd.DataFrame(contract_encoded, columns = labels, index = telco.index)

In [ ]:
contract_encoded_df.head()

In [15]:
# now that all the one hot encoded dataframes are ready to go, we need to join them to the orginal dataframe

telco = telco.join([payment_encoded_df, internet_encoded_df, contract_encoded_df])

In [ ]:
telco.head()

In [ ]:
telco.shape

In [ ]:
telco.columns

In [16]:
telco = telco.drop(columns=['contract_type', 'internet_service_type', 'payment_type'])

In [ ]:
telco.head()

In [17]:
pd.set_option('display.max_columns', None)

In [ ]:
telco.sample(10)

In [ ]:
telco.shape

In [ ]:
encoder = LabelEncoder()
encoder.fit_transform(telco[['online_security']])

telco.online_security = encoder.transform(telco[['online_security']])

In [ ]:
telco.head()

In [36]:
def X_label_encode(df):
    le = LabelEncoder()
    df['online_security'] = le.fit_transform(df.online_security)
    df['online_backup'] = le.fit_transform(df.online_backup)
    df['device_protection'] = le.fit_transform(df.device_protection)
    df['tech_support'] = le.fit_transform(df.tech_support)
    df['streaming_tv'] = le.fit_transform(df.streaming_tv)
    df['streaming_movies'] = le.fit_transform(df.streaming_movies)
    df['paperless_billing'] = le.fit_transform(df.paperless_billing)
    # df['churn'] = le.fit_transform(df.churn)
    return df

In [37]:
def y_label_encode(df):
    le = LabelEncoder()
    df['churn'] = le.fit_transform(df.churn)
    return df

In [ ]:
pd.crosstab(telco.online_backup, telco.online_security)

In [ ]:
telco = label_encode(telco)

In [ ]:
telco.sample(20)

In [40]:
def get_telco_splits(telco):
    '''
    Returns X and y for train, validate and test datasets
    '''
    # don't blow away our original data
    telco = telco.copy()
    
#     # ignore warnings just for this block
#     with warnings.catch_warnings():
#         warnings.simplefilter('ignore')
#         scaler, encoder, train, test = prepare_walkthrough.prep_iris(iris)
    
    # Which features are we going to look at?
    train = telco
    test = telco

    # validate data split
    train, test = sklearn.model_selection.train_test_split(train, train_size=.80, random_state=123)
    train, validate = sklearn.model_selection.train_test_split(train, train_size=.80, random_state=123)

    # split into X and y
    X_train, y_train = train.drop(columns='churn'), train[['churn']]
    X_validate, y_validate = validate.drop(columns='churn'), validate[['churn']]
    X_test, y_test = test.drop(columns='churn'), test[['churn']]
    
    X_train = X_label_encode(X_train)
    X_validate = X_label_encode(X_validate)
    X_test = X_label_encode(X_test)
    
    y_train = y_label_encode(y_train)
    y_validate = y_label_encode(y_validate)
    y_test = y_label_encode(y_test)

    return X_train, y_train, X_validate, y_validate, X_test, y_test

In [41]:
X_train, y_train, X_validate, y_validate, X_test, y_test = get_telco_splits(telco)

In [42]:
print('   train: %d rows' % X_train.shape[0])
print('validate: %d rows' % X_validate.shape[0])
print('    test: %d rows' % X_test.shape[0])

   train: 4500 rows
validate: 1125 rows
    test: 1407 rows


In [43]:
X_train.head()

,gender,senior_citizen,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,tenure_years,phone_and_multi_line,partner_and_dependents,Electronic check,Mailed check,Bank transfer (automatic),Credit card (automatic),DSL,Fiber optic,None,Month-to-month,One year,Two year
customer_id,,,,,,,,,,,,,,,,,,,,,,,,
3714-JTVOV,Female,1,2,0,0,0,0,0,1,74.15,3229.40,3.500000,1,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3049-SOLAY,Female,0,0,0,0,0,2,2,1,95.20,292.85,0.250000,2,1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
5035-PGZXH,Female,0,0,2,2,0,2,2,1,106.80,5914.40,4.666667,2,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1051-EQPZR,Female,0,1,1,1,1,1,1,0,19.60,780.25,3.666667,1,3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
8755-OGKNA,Female,0,1,1,1,1,1,1,0,19.50,1167.60,4.750000,1,3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [45]:
y_train.head()

,churn
customer_id,
3714-JTVOV,1
3049-SOLAY,1
5035-PGZXH,0
1051-EQPZR,0
8755-OGKNA,0
